Lecture 3
===========================================================================

Convexity and Hessians
------------

In [ ]:
from sympy import *
init_printing()

Let's find the derivatives of a few functions

In [ ]:
x = Symbol('x', real=True)
functions = (('a', 3*x**2),
             ('b', 2*x),
             ('c', -5*x**2),
             ('d', 2*x**2 - x**3))
for (c, f) in functions:
    derivative = diff(f, x, 2)
    positivex = solve(derivative>0, x)
    print c, str(derivative), positivex

We can also evaluate symbolic expressions of multiple variables

In [ ]:
x1, x2 = symbols('x1 x2')
f = 2*x1**3 - 3*x1*x2 + 2*x2**3
f

For example f(1,2)

In [ ]:
f.subs({x1: x + 2, x2: 2})

Digression 1: dictionaries
===============

You should be familiar with lists:

In [ ]:
lst = [1, 2, 3]

We can retrieve elements from the list by _indexing_ it

In [ ]:
lst[1]

A dictionary gives us a container like a list, but the indexes can be much more general, not just numbers but strings or sympy variables (and a whole host of other types)

In [ ]:
dic = {'a': 100, 2: 45, 100: 45}

In [ ]:
dic['a']

When we solve and equation in sympy, the result is a dictionary

In [ ]:
eqs = [x1 - 3 - x2, 
       x2 + x1]
ans = solve(eqs, [x1, x2])

In [ ]:
type(ans)

Digression 2: List comprehensions
================

The following pattern is very common in code, where we start with an empty list and append to it by processing another list.

In [ ]:
v = []
for i in [2, 1, 3, 4]:
    if i > 2:
        v.append(i**2)
v

In fact, it is so common that there is a shorthand in Python which does exactly that called a _list comprehension_

In [ ]:
[i**2 for i in [2, 1, 3, 4] if i > 2]

This gives us an easy way to substitute the solution of our previous set of equtions into each equation:

In [ ]:
[eq.subs(ans) for eq in eqs]

And take the partial derivative w.r.t. each variable

In [ ]:
variables = [x1, x2]
df = [f.diff(variable, 1) for variable in variables]
df

We can use this to build a function to calculate the gradient vector:

In [ ]:
f.subs({x1: x + 2, x2: 2})

In [ ]:
def gradient(f, variables):
    return Matrix([f.diff(variable) for variable in variables])

In [ ]:
gradient(f, variables)

Second order derivative

In [ ]:
df2 = [f.diff(variable, 2) for variable in variables]
df2

Partial derivative w.r.t. one variable then the next

In [ ]:
dfx1x2 = [df[i].diff(variable) for variable in reversed(variables)]
dfx1x2

Remember the Hessian is the matrix of repeated partial differentials:

$$\begin{bmatrix} \frac{\partial^2 f}{\partial x_{1}^2} & \frac{\partial^2 f}{\partial x_{1} \partial x_{2}} \\ \frac{\partial^2 f}{\partial x_{2} \partial x_{1}} & \frac{\partial^2 f}{\partial x_{2}^2} \end{bmatrix}$$

Luckily, sympy supplies a function to calculate the Hessian

In [ ]:
H = hessian(f, variables)
H

We can still evaluate these expressions at a specific point

In [ ]:
H.subs({x1: 1, x2: 2})

In [ ]:
f2 = 2*x1**2 + 2*x1*x2 + 1.5*x2**2 + 7*x1 + 8*x2 + 24
H2 = hessian(f2, [x1, x2])
H2

Positive definiteness is the matrix analogue to positiveness in scalars. Observe these definitions:

$x^THx \geq 0 \forall x \iff H~\text{positive semidefinite}$ similar to $h \geq 0 \iff h~\text{positive}$

$x^THx > 0 \forall x \neq 0 \iff H~\text{positive definite}$ similar to $h > 0 \iff h~\text{strictly positive}$

$x^THx < 0 \forall x \neq 0 \iff H~\text{negative definite}$ similar to $h < 0 \iff h~\text{strictly negative}$

$x^THx \leq 0 \forall x \iff H~\text{positive semidefinite}$ similar to $h \leq 0 \iff h~\text{negative}$

There are several ways to determine whether a matrix is positive definite.

The first involves evaluating the leading principal minors. The minors are the determinants of matrices obtained by removing a certain number of rows and columns. If the rows and columns which are removed have the same indices, the minor is called a principal minor. If the rows and columns removed are also from the beginning to some point $i$, it is known as a leading minor.

In [ ]:
def leading_minors(A):
    rows, cols = A.shape
    return (A[:n+1, :n+1].det() for n in range(rows))

In [ ]:
list(leading_minors(H2))

In [ ]:
H2.berkowitz_minors()

In [ ]:
def posdef_minor(A):
    return all(m >= 0 for m in leading_minors(A))

In [ ]:
posdef_minor(H2)

The second method is to check if all the eigenvalues are positive

In [ ]:
H2

In [ ]:
EV = H2.eigenvals()
EVs = [N(i,3) for i in EV]
EVs

In [ ]:
def posdef_eigen(A):
    return all(EV >= 0 for EV in A.eigenvals())

In [ ]:
posdef_eigen(H2)

So what's the difference between these methods? They will all work, but some take longer than others.

In [ ]:
size = 20

In [ ]:
%%timeit
H2 = randMatrix(size)
posdef_minor(H2)

In [ ]:
%%timeit
H2 = randMatrix(size)
posdef_eigen(H2)